In [6]:
# RAW IQ single-frame training + m-frame query late fusion (m=288)
# Block construction (per TX):
#   - Load ALL frames from ALL files under the same TX
#   - Shuffle all frames within TX
#   - Split into blocks of size m=288 sequentially after shuffling
# Train:
#   - single-frame training (input: (seg_len,2))
# Split:
#   - train/val/test split by BLOCK (no leakage)
# Eval:
#   - late fusion within block: mean logits over m frames

import os
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold
from datetime import datetime
from tqdm import tqdm
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import h5py
import matplotlib.pyplot as plt
import seaborn as sns

# ================= 参数设置 =================
data_path = "E:/rf_datasets/"  # 存放所有.mat文件的文件夹路径

# 信号参数
SNR_dB = 20           # 信噪比（AWGN）
fs = 5e6            # 采样率 Hz
fc = 5.9e9           # 载波频率 Hz
v = 120              # 速度：此处按 m/s（若是 km/h，请自行改 compute_doppler_shift）

apply_doppler = True
apply_awgn = True
normalize_power = False   # 如需与XFR一致，可设 True

# block/query参数
m_query = 288
block_batch_size = 8      # block级评估 batch

# 模型超参数（Transformer 单帧）
raw_input_dim = 2
model_dim = 64
num_heads = 8
num_layers = 2
dropout = 0.1

# 训练参数
batch_size = 64
num_epochs = 100
learning_rate = 5e-4
patience = 5
n_splits = 5
weight_decay = 1e-4
seed = 42

# ================= 随机种子 =================
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(seed)

# ================= 多普勒和AWGN处理函数 =================
def compute_doppler_shift(v_mps, fc_hz):
    c = 3e8
    v_mps = v_mps/3.6
    return (v_mps / c) * fc_hz

def apply_doppler_shift(signal_c, fd_hz, fs_hz):
    t = np.arange(signal_c.shape[-1], dtype=np.float64) / fs_hz
    doppler_phase = np.exp(1j * 2 * np.pi * fd_hz * t)
    return signal_c * doppler_phase

def add_awgn(signal_c, snr_db):
    signal_power = np.mean(np.abs(signal_c)**2)
    noise_power = signal_power / (10**(snr_db/10))
    noise = np.sqrt(noise_power/2) * (np.random.randn(*signal_c.shape) + 1j*np.random.randn(*signal_c.shape))
    return signal_c + noise

def power_normalize(signal_c, eps=1e-12):
    return signal_c / (np.sqrt(np.mean(np.abs(signal_c)**2)) + eps)

# ================= 兼容读取 v7.3 HDF5 dmrs 与 txID =================
def read_tx_id_str(rfDataset):
    txID_uint16 = rfDataset['txID'][:].flatten()
    tx_id = ''.join(chr(int(c)) for c in txID_uint16 if int(c) != 0)
    return tx_id

def read_dmrs_complex(rfDataset):
    dmrs = rfDataset['dmrs']
    # 情形1：Group，含 real/imag
    if isinstance(dmrs, h5py.Group):
        real = dmrs['real'][:]
        imag = dmrs['imag'][:]
        return real + 1j * imag
    # 情形2：Dataset，compound dtype
    arr = dmrs[:]
    if hasattr(arr, "dtype") and arr.dtype.fields is not None and ('real' in arr.dtype.fields) and ('imag' in arr.dtype.fields):
        return arr['real'] + 1j * arr['imag']
    # 情形3：已是 complex 或其它
    return arr

# ================= 核心：按TX加载->TX内shuffle->切block =================
def load_blocks_shuffle_within_tx(
    mat_folder,
    m_query=288,
    apply_doppler=False,
    apply_awgn=False,
    snr_db=0,
    fs=20e6,
    fc=2.4e9,
    v_mps=120,
    normalize_power=False,
    seed=42
):
    mat_files = glob.glob(os.path.join(mat_folder, "*.mat"))
    if len(mat_files) == 0:
        raise RuntimeError(f"在 {mat_folder} 未找到 .mat 文件")
    print(f"[INFO] 共找到 {len(mat_files)} 个 .mat 文件")

    # 1) 先按 TX 分组文件
    tx_to_files = {}
    for fp in tqdm(mat_files, desc="扫描 txID"):
        with h5py.File(fp, "r") as f:
            rfDataset = f["rfDataset"]
            tx_id = read_tx_id_str(rfDataset)
        tx_to_files.setdefault(tx_id, []).append(fp)

    tx_list = sorted(tx_to_files.keys())
    label_to_idx = {tx: i for i, tx in enumerate(tx_list)}
    print(f"[INFO] TX数量: {len(tx_list)} | label_to_idx={label_to_idx}")

    fd = compute_doppler_shift(v_mps, fc)
    print(f"[INFO] v={v_mps} m/s => fd={fd:.2f} Hz | SNR={snr_db} dB | m={m_query}")

    rng = np.random.default_rng(seed)

    X_blocks_all = []
    y_blocks_all = []

    # 2) 对每个 TX：读所有文件所有帧 -> 预处理 -> shuffle -> 切 block
    for tx in tx_list:
        files = sorted(tx_to_files[tx])
        label_idx = label_to_idx[tx]

        frames_list = []
        seg_len_ref = None

        print(f"\n[INFO] TX={tx} | files={len(files)} | 读取所有帧并预处理...")
        for fp in tqdm(files, desc=f"TX={tx} files"):
            with h5py.File(fp, "r") as f:
                rfDataset = f["rfDataset"]
                dmrs_complex = read_dmrs_complex(rfDataset)  # (N, seg_len) complex

            if dmrs_complex.ndim != 2:
                raise RuntimeError(f"dmrs维度异常: {dmrs_complex.shape} | file={fp}")

            N, seg_len = dmrs_complex.shape
            if seg_len_ref is None:
                seg_len_ref = seg_len
            elif seg_len != seg_len_ref:
                raise RuntimeError(f"同一TX下 seg_len 不一致: {seg_len_ref} vs {seg_len} | file={fp}")

            processed = np.empty((N, seg_len, 2), dtype=np.float32)
            for i in range(N):
                sig = dmrs_complex[i, :]

                if normalize_power:
                    sig = power_normalize(sig)

                if apply_doppler:
                    sig = apply_doppler_shift(sig, fd, fs)

                if apply_awgn:
                    sig = add_awgn(sig, snr_db)

                processed[i, :, 0] = sig.real.astype(np.float32)
                processed[i, :, 1] = sig.imag.astype(np.float32)

            frames_list.append(processed)

        X_tx = np.concatenate(frames_list, axis=0)  # (N_total, seg_len, 2)
        N_total = X_tx.shape[0]

        # 3) TX内随机打乱
        perm = rng.permutation(N_total)
        X_tx = X_tx[perm]

        # 4) 切 blocks（不足m的尾巴丢弃）
        num_blocks = N_total // m_query
        if num_blocks == 0:
            print(f"[WARN] TX={tx} 总帧数 {N_total} < m={m_query}，无法构造block，跳过")
            continue

        X_tx_use = X_tx[: num_blocks * m_query]
        X_tx_blocks = X_tx_use.reshape(num_blocks, m_query, seg_len_ref, 2)  # (Btx, m, seg_len, 2)

        X_blocks_all.append(X_tx_blocks)
        y_blocks_all.append(np.full((num_blocks,), label_idx, dtype=np.int64))

        print(f"[INFO] TX={tx} | total_frames={N_total} | blocks={num_blocks} | seg_len={seg_len_ref}")

    if len(X_blocks_all) == 0:
        raise RuntimeError("未构造出任何 blocks，请检查数据与 m_query 设置。")

    X_blocks = np.concatenate(X_blocks_all, axis=0).astype(np.float32)  # (Nblk, m, seg_len, 2)
    y_blocks = np.concatenate(y_blocks_all, axis=0).astype(np.int64)    # (Nblk,)
    print(f"\n[INFO] 总 blocks={X_blocks.shape[0]} | X_blocks={X_blocks.shape} | y_blocks={y_blocks.shape}")
    return X_blocks, y_blocks, label_to_idx

# ================= Dataset：从blocks生成单帧样本（避免reshape复制） =================
class FrameFromBlocksDataset(Dataset):
    def __init__(self, X_blocks_t, y_blocks_t, block_indices, m_query):
        self.X = X_blocks_t
        self.y = y_blocks_t
        self.block_indices = np.array(block_indices, dtype=np.int64)
        self.m = m_query
        self.num_blocks = len(self.block_indices)

    def __len__(self):
        return self.num_blocks * self.m

    def __getitem__(self, idx):
        b = idx // self.m
        t = idx % self.m
        blk_idx = int(self.block_indices[b])
        x = self.X[blk_idx, t]     # (seg_len,2)
        y = self.y[blk_idx]        # scalar
        return x, y

class BlockDataset(Dataset):
    def __init__(self, X_blocks_t, y_blocks_t, block_indices):
        self.X = X_blocks_t
        self.y = y_blocks_t
        self.block_indices = np.array(block_indices, dtype=np.int64)

    def __len__(self):
        return len(self.block_indices)

    def __getitem__(self, idx):
        blk_idx = int(self.block_indices[idx])
        return self.X[blk_idx], self.y[blk_idx]  # (m, seg_len,2), label

# ================= Transformer 单帧模型 =================
class SignalTransformer(nn.Module):
    def __init__(self, raw_input_dim, model_dim, num_heads, num_layers, num_classes, dropout=0.1):
        super().__init__()
        self.embedding = nn.Linear(raw_input_dim, model_dim)
        encoder_layer = TransformerEncoderLayer(
            d_model=model_dim, nhead=num_heads, dropout=dropout, batch_first=True
        )
        self.encoder = TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(model_dim, num_classes)

    def forward(self, x):
        # x: (B, seg_len, 2)
        x = self.embedding(x)
        x = self.encoder(x)
        x = x[:, -1, :]  # 最后时间步
        x = self.fc(x)
        return x

# ================= 评估：m-frame late fusion（mean logits） =================
@torch.no_grad()
def evaluate_late_fusion(model, block_loader, device, num_classes):
    model.eval()
    correct, total = 0, 0
    all_labels, all_preds = [], []

    for blocks, labels in block_loader:
        blocks = blocks.to(device)  # (B, m, seg_len, 2)
        labels = labels.to(device)  # (B,)
        B, M, L, C = blocks.shape

        flat = blocks.reshape(B * M, L, C)     # (B*M, seg_len, 2)
        logits = model(flat)                   # (B*M, K)
        logits = logits.reshape(B, M, num_classes)
        fused_logits = logits.mean(dim=1)      # (B, K)
        preds = torch.argmax(fused_logits, dim=1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

    acc = 100.0 * correct / max(total, 1)
    cm = confusion_matrix(all_labels, all_preds, labels=range(num_classes))
    return acc, cm

def moving_average(x, w=5):
    x = np.array(x, dtype=np.float64)
    if len(x) == 0:
        return x
    w = max(1, min(w, len(x)))
    return np.convolve(x, np.ones(w), 'valid') / w

def plot_loss_curves(train_losses, val_losses, save_path):
    plt.figure(figsize=(8,4))
    plt.plot(moving_average(train_losses), label="Train Loss")
    plt.plot(moving_average(val_losses), label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def plot_confusion_matrix(cm, save_path=None, title="Confusion Matrix"):
    plt.figure(figsize=(8, 6))
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.ylabel('真实类别')
    plt.xlabel('预测类别')
    if save_path:
        plt.savefig(save_path)
    plt.close()

# ================= 主流程 =================
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[INFO] device={device}")

    # 1) 构造 blocks（TX内shuffle后切block）
    X_blocks, y_blocks, label_to_idx = load_blocks_shuffle_within_tx(
        data_path,
        m_query=m_query,
        apply_doppler=apply_doppler,
        apply_awgn=apply_awgn,
        snr_db=SNR_dB,
        fs=fs,
        fc=fc,
        v_mps=v,
        normalize_power=normalize_power,
        seed=seed
    )

    num_blocks, M, seg_len, two = X_blocks.shape
    num_classes = len(label_to_idx)
    print(f"[INFO] num_blocks={num_blocks} | m={M} | seg_len={seg_len} | classes={num_classes}")

    X_blocks_t = torch.from_numpy(np.ascontiguousarray(X_blocks)).float()
    y_blocks_t = torch.from_numpy(np.ascontiguousarray(y_blocks)).long()

    # 2) 按 block 划分 train/test（25%）
    block_indices = np.arange(num_blocks)
    train_val_blocks, test_blocks = train_test_split(
        block_indices, test_size=0.25, random_state=seed, stratify=y_blocks
    )

    # 固定 test block loader（late fusion评估）
    test_block_ds = BlockDataset(X_blocks_t, y_blocks_t, test_blocks)
    test_block_loader = DataLoader(test_block_ds, batch_size=block_batch_size, shuffle=False)

    # 3) K折（按block stratified）
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    y_train_val = y_blocks[train_val_blocks]

    # 结果目录
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    out_dir = os.path.join(
        os.getcwd(),
        "training_results",
        f"{timestamp}_RAWIQ_TXshuffle_Block_m{m_query}_SNR{SNR_dB}dB_fd{int(compute_doppler_shift(v,fc))}_Transformer"
    )
    os.makedirs(out_dir, exist_ok=True)
    results_file = os.path.join(out_dir, "results.txt")

    with open(results_file, "w") as f:
        f.write("=== RAW IQ single-frame training + m-frame late fusion (TX-internal shuffle blocks) ===\n")
        f.write(f"time: {timestamp}\n")
        f.write(f"classes: {num_classes}\n")
        f.write(f"m_query: {m_query}\n")
        f.write(f"seg_len: {seg_len}\n")
        f.write(f"SNR: {SNR_dB} dB | doppler={apply_doppler} | awgn={apply_awgn}\n")
        f.write(f"v: {v} (m/s assumed) | fd: {compute_doppler_shift(v,fc):.2f} Hz\n")
        f.write(f"normalize_power: {normalize_power}\n")
        f.write(f"train_val blocks: {len(train_val_blocks)} | test blocks: {len(test_blocks)}\n")

    fold_val_fused = []
    fold_test_fused = []

    for fold, (tr_idx, va_idx) in enumerate(skf.split(train_val_blocks, y_train_val), start=1):
        print(f"\n====== Fold {fold}/{n_splits} ======")

        fold_train_blocks = train_val_blocks[tr_idx]
        fold_val_blocks = train_val_blocks[va_idx]

        # 单帧训练/验证集（由blocks动态展开）
        train_frame_ds = FrameFromBlocksDataset(X_blocks_t, y_blocks_t, fold_train_blocks, m_query)
        val_frame_ds = FrameFromBlocksDataset(X_blocks_t, y_blocks_t, fold_val_blocks, m_query)

        train_loader = DataLoader(train_frame_ds, batch_size=batch_size, shuffle=True, drop_last=True)
        val_loader = DataLoader(val_frame_ds, batch_size=batch_size, shuffle=False, drop_last=False)

        # block级验证集（late fusion）
        val_block_ds = BlockDataset(X_blocks_t, y_blocks_t, fold_val_blocks)
        val_block_loader = DataLoader(val_block_ds, batch_size=block_batch_size, shuffle=False)

        model = SignalTransformer(raw_input_dim, model_dim, num_heads, num_layers, num_classes, dropout).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

        best_val_fused = -1.0
        best_state = None
        patience_cnt = 0

        train_losses = []
        val_losses = []

        for epoch in range(1, num_epochs + 1):
            # ---- train single-frame ----
            model.train()
            running_loss = 0.0
            correct, total = 0, 0

            for x, y in tqdm(train_loader, desc=f"Fold{fold} Ep{epoch}", leave=False):
                x = x.to(device)
                y = y.to(device)

                optimizer.zero_grad()
                logits = model(x)
                loss = criterion(logits, y)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                pred = torch.argmax(logits, dim=1)
                correct += (pred == y).sum().item()
                total += y.size(0)

            train_loss = running_loss / max(len(train_loader), 1)
            train_acc = 100.0 * correct / max(total, 1)
            train_losses.append(train_loss)

            # ---- val loss (single-frame) ----
            model.eval()
            running_vloss = 0.0
            v_correct, v_total = 0, 0
            with torch.no_grad():
                for x, y in val_loader:
                    x = x.to(device)
                    y = y.to(device)
                    logits = model(x)
                    loss = criterion(logits, y)
                    running_vloss += loss.item()
                    pred = torch.argmax(logits, dim=1)
                    v_correct += (pred == y).sum().item()
                    v_total += y.size(0)

            val_loss = running_vloss / max(len(val_loader), 1)
            val_acc_single = 100.0 * v_correct / max(v_total, 1)
            val_losses.append(val_loss)

            # ---- val fused acc (primary) ----
            val_acc_fused, _ = evaluate_late_fusion(model, val_block_loader, device, num_classes)

            log = (f"Fold {fold} Ep {epoch:03d} | "
                   f"TrainLoss {train_loss:.4f} TrainAcc(single) {train_acc:.2f}% | "
                   f"ValLoss {val_loss:.4f} ValAcc(single) {val_acc_single:.2f}% | "
                   f"ValAcc(fused,m={m_query}) {val_acc_fused:.2f}%")
            print(log)
            with open(results_file, "a") as f:
                f.write(log + "\n")

            # early stopping on fused val acc
            if val_acc_fused > best_val_fused + 1e-6:
                best_val_fused = val_acc_fused
                best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
                patience_cnt = 0
            else:
                patience_cnt += 1
                if patience_cnt >= patience:
                    print(f"[INFO] Early stop: {patience} epochs no improvement on fused val acc.")
                    break

            scheduler.step()

        # restore best
        if best_state is not None:
            model.load_state_dict(best_state)

        # final val/test fused
        val_acc_fused, val_cm = evaluate_late_fusion(model, val_block_loader, device, num_classes)
        test_acc_fused, test_cm = evaluate_late_fusion(model, test_block_loader, device, num_classes)

        fold_val_fused.append(val_acc_fused)
        fold_test_fused.append(test_acc_fused)

        plot_loss_curves(train_losses, val_losses, os.path.join(out_dir, f"loss_fold{fold}.png"))
        plot_confusion_matrix(val_cm, os.path.join(out_dir, f"cm_val_fused_fold{fold}.png"),
                              title=f"Val CM (Fused, m={m_query})")
        plot_confusion_matrix(test_cm, os.path.join(out_dir, f"cm_test_fused_fold{fold}.png"),
                              title=f"Test CM (Fused, m={m_query})")

        ckpt_path = os.path.join(out_dir, f"best_model_fold{fold}.pth")
        torch.save(model.state_dict(), ckpt_path)

        msg = f"[RESULT] Fold {fold} | Val(fused)={val_acc_fused:.2f}% | Test(fused)={test_acc_fused:.2f}% | ckpt={ckpt_path}"
        print(msg)
        with open(results_file, "a") as f:
            f.write(msg + "\n")

    print("\n====== Finished ======")
    print(f"Val(fused)  mean±std: {np.mean(fold_val_fused):.2f}% ± {np.std(fold_val_fused):.2f}%")
    print(f"Test(fused) mean±std: {np.mean(fold_test_fused):.2f}% ± {np.std(fold_test_fused):.2f}%")

    with open(results_file, "a") as f:
        f.write(f"\nAll folds Val(fused) mean±std: {np.mean(fold_val_fused):.2f}% ± {np.std(fold_val_fused):.2f}%\n")
        f.write(f"All folds Test(fused) mean±std: {np.mean(fold_test_fused):.2f}% ± {np.std(fold_test_fused):.2f}%\n")

    print(f"[INFO] results saved to: {out_dir}")

if __name__ == "__main__":
    main()


[INFO] device=cuda
[INFO] 共找到 72 个 .mat 文件


扫描 txID: 100%|██████████| 72/72 [00:00<00:00, 3440.77it/s]


[INFO] TX数量: 9 | label_to_idx={'001': 0, '002': 1, '003': 2, '004': 3, '005': 4, '006': 5, '007': 6, '008': 7, '009': 8}
[INFO] v=120 m/s => fd=655.56 Hz | SNR=20 dB | m=288

[INFO] TX=001 | files=8 | 读取所有帧并预处理...


TX=001 files: 100%|██████████| 8/8 [00:00<00:00,  8.52it/s]


[INFO] TX=001 | total_frames=23992 | blocks=83 | seg_len=288

[INFO] TX=002 | files=8 | 读取所有帧并预处理...


...

Fold 5 Ep 011 | TrainLoss 2.0275 TrainAcc(single) 22.09% | ValLoss 2.0084 ValAcc(single) 22.83% | ValAcc(fused,m=288) 100.00%


Fold 5 Ep 012 | TrainLoss 2.0207 TrainAcc(single) 22.31% | ValLoss 2.0144 ValAcc(single) 22.41% | ValAcc(fused,m=288) 100.00%


Fold 5 Ep 013 | TrainLoss 2.0163 TrainAcc(single) 22.46% | ValLoss 1.9966 ValAcc(single) 23.31% | ValAcc(fused,m=288) 100.00%


Fold 5 Ep 014 | TrainLoss 2.0127 TrainAcc(single) 22.79% | ValLoss 2.0084 ValAcc(single) 22.60% | ValAcc(fused,m=288) 100.00%


Fold 5 Ep 015 | TrainLoss 2.0082 TrainAcc(single) 22.91% | ValLoss 1.9926 ValAcc(single) 23.64% | ValAcc(fused,m=288) 100.00%
[INFO] Early stop: 5 epochs no improvement on fused val acc.
[RESULT] Fold 5 | Val(fused)=100.00% | Test(fused)=99.47% | ckpt=d:\Program\MW-RFF\MW-RFF\training_results\2026-01-21_17-50-12_RAWIQ_TXshuffle_Block_m288_SNR20dB_fd655_Transformer\best_model_fold5.pth

====== Finished ======
Val(fused)  mean±std: 100.00% ± 0.00%
Test(fused) mean±std: 99.68% ± 0.43%
[INFO] results saved to: d:\Program\MW-RFF\MW-RFF\training_results\2026-01-21_17-50-12_RAWIQ_TXshuffle_Block_m288_SNR20dB_fd655_Transformer
